In [2]:
import torch
from torch import nn 
import numpy as np
import gym
import matplotlib.pyplot as plt
import time

In [3]:
class CEM(nn.Module):
    def __init__(self, state_dim, action_n):
        super().__init__()
        self.state_dim = state_dim
        self.action_n = action_n
        
        self.network = nn.Sequential(
            nn.Linear(self.state_dim, 100), 
            nn.ReLU(), 
            nn.Linear(100, self.action_n)
        )
        
        self.softmax = nn.Softmax()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.01)
        self.loss = nn.CrossEntropyLoss()
        
    def forward(self, _input):
        return self.network(_input) 
    
    def get_action(self, state):
        state = torch.FloatTensor(state)
        logits = self.forward(state)
        action_prob = self.softmax(logits).detach().numpy()
        action = np.random.choice(self.action_n, p=action_prob)
        return action
    
    def update_policy(self, elite_trajectories):
        elite_states = []
        elite_actions = []
        for trajectory in elite_trajectories:
            elite_states.extend(trajectory['states'])
            elite_actions.extend(trajectory['actions'])
        elite_states = torch.FloatTensor(elite_states)
        elite_actions = torch.LongTensor(elite_actions)
        
        loss = self.loss(self.forward(elite_states), elite_actions)
        loss.backward()
        self.optimizer.step()
        self.optimizer.zero_grad()
        
        
def get_trajectory(env, agent, trajectory_len, visualize=False):
    trajectory = {'states':[], 'actions': [], 'total_reward': 0}
    
    state = env.reset()
    trajectory['states'].append(state)
    
    for _ in range(trajectory_len):
        
        action = agent.get_action(state)
        trajectory['actions'].append(action)
        
        state, reward, done, _ = env.step(action)
        trajectory['total_reward'] += reward
        
        if done:
            break
            
        if visualize:
            env.render()
            
        trajectory['states'].append(state)
            
    return trajectory

def get_elite_trajectories(trajectories, q_param):
    total_rewards = [trajectory['total_reward'] for trajectory in trajectories]
    quantile = np.quantile(total_rewards, q=q_param) 
    return [trajectory for trajectory in trajectories if trajectory['total_reward'] > quantile]


env = gym.make('CartPole-v1')
state_dim = 4
action_n = 2

agent = CEM(state_dim, action_n)
episode_n = 50
trajectory_n = 20
trajectory_len = 500
q_param = 0.8

for episode in range(episode_n):
    start = time.time()
    trajectories = [get_trajectory(env, agent, trajectory_len) for _ in range(trajectory_n)]
    
    mean_total_reward = np.mean([trajectory['total_reward'] for trajectory in trajectories])
    print(f'episode: {episode}, mean_total_reward = {mean_total_reward}')
    
    elite_trajectories = get_elite_trajectories(trajectories, q_param)
    
    if len(elite_trajectories) > 0:
        agent.update_policy(elite_trajectories)
    print('iteration_time: ', time.time() - start)
# get_trajectory(env, agent, trajectory_len, visualize=True)

g:\PycharmProjects\DRL\env\Lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\denis\AppData\Local\Temp\ipykernel_16724\2406726340.py:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  elite_states = torch.FloatTensor(elite_states)


episode: 0, mean_total_reward = 28.4
iteration_time:  0.03804278373718262
episode: 1, mean_total_reward = 24.55
iteration_time:  0.028577089309692383
episode: 2, mean_total_reward = 30.85
iteration_time:  0.03633928298950195
episode: 3, mean_total_reward = 30.55
iteration_time:  0.03494381904602051
episode: 4, mean_total_reward = 35.35
iteration_time:  0.03905773162841797
episode: 5, mean_total_reward = 46.0
iteration_time:  0.05386209487915039
episode: 6, mean_total_reward = 52.15
iteration_time:  0.05909156799316406
episode: 7, mean_total_reward = 45.15
iteration_time:  0.05202150344848633
episode: 8, mean_total_reward = 51.05
iteration_time:  0.05845952033996582
episode: 9, mean_total_reward = 57.3
iteration_time:  0.06869053840637207
episode: 10, mean_total_reward = 64.3
iteration_time:  0.07346439361572266
episode: 11, mean_total_reward = 66.2
iteration_time:  0.07727622985839844
episode: 12, mean_total_reward = 65.45
iteration_time:  0.0738224983215332
episode: 13, mean_total_rew